In [4]:
#import things

import cv2
import numpy  as np
from collections import deque
import time

In [5]:
#define things

points = deque(maxlen = 1000)
red_upper = (150,150,50)
red_lower = (200,255,150)
blackboard = np.zeros((512,512), np.uint8)


In [8]:
start_time = time.time()


cap = cv2.VideoCapture(0)
points
    
while time.time() < start_time + 10:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    #hsv
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #red
    red = cv2.inRange(hsv, red_upper, red_lower)
    #red median blur
    red_median = cv2.medianBlur(red, 15)
    #find contours
    _, contours, _ = cv2.findContours(red_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    try:
        if len(contours) > 0:
            contour_list = sorted(contours, key = cv2.contourArea, reverse = True)
            contour = contour_list[0]
            #find centroid
            M = cv2.moments(contour) #M will be a dict
            centroid_x = int(M["m10"]/M["m00"])
            centroid_y = int(M["m01"]/M["m00"])
            centroid = (centroid_x, centroid_y)
            #append centroid in points
            points.appendleft(centroid)
            #draw circle
            ((x,y), r) = cv2.minEnclosingCircle(contour)
            cv2.circle(frame, (int(x),int(y)), int(r), (0,255,0), 2)
    except ZeroDivisionError:
        pass
            
    for i in range(1, len(points)):
        cv2.line(frame, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (0,0,0), 2)
        cv2.line(blackboard, tuple(map(int,points[i])), tuple(map(int,points[i-1])), (255,255,255), 8)

    cv2.imshow("frame", frame)
    cv2.imshow("blackboard", blackboard)
    
    if cv2.waitKey(1) &  0xFF == ord("q"):
        break

    
blackboard_median = cv2.medianBlur(blackboard, 15)
alphabet = cv2.resize(blackboard_median, (28,28))
alphabet = np.array(alphabet)
alphabet = alphabet.astype("float32")/255


    
points = deque(maxlen = 1000)
blackboard = np.zeros((512,512), np.uint8)

cv2.destroyAllWindows()
cap.release()

In [18]:
write()

KeyboardInterrupt: 